## 一、准备工作：文本预处理
**部分的原始文本**：
```
The Time Traveller (for so it will be convenient to speak of him)
was expounding a recondite matter to us. His grey eyes shone and
twinkled, and his usually pale face was flushed and animated. The
fire burned brightly, and the soft radiance of the incandescent
lights in the lilies of silver caught the bubbles that flashed and
passed in our glasses. Our chairs, being his patents, embraced and
```

**步骤1：文本清洗**
* 将每一个行文本转小写、移除标点符号、只保留字母和空格后，放入到一个 token 中，形成一个 tokens 列表
* tokens 展平为一个字符列表：['t', 'h', 'e', ' ', 't', 'i', 'm', 'e', ...]

**步骤2：建立词汇表**
* 词汇表：[' ', 'a', 'b', 'c', ..., 'z']
* vocab_size = 27
* 索引映射：' ' → 0, 'a' → 1, 'b' → 2, ..., 'z' → 26
* 得到索引序列：`corpus = [19, 7, 4, 26, 19, 8, 12, 4, 26, ...]`

此时 corpus 的 shape（如果看成张量）是：(num_chars,)

## 二、关于 inputs 输入
在 D2L 中，RNN 的输入 `inputs` 是一个形状为 `(num_steps, batch_size, vocab_size)` 的张量
* num_steps：时间步（第 0 个字符、第 1 个字符……第 num_steps-1 个字符）
* batch_size：并行的样本条数（同一时间步同时处理多少条序列）
* vocab_size：每个字符的 one-hot 向量长度（27 维，空格 + 26 个字母）

所以在时间步 t，RNN 实际拿到的是：
> $X_t$ 的 shape = (batch_size, vocab_size) \
> 也就是 “这一时刻 batch 中每条序列的当前字符（one-hot）”。

### 三、训练样本的本质：用 X 预测 Y（整体右移 1 位）
无论随机采样还是顺序分区，语言模型训练都在做同一件事：
* 输入序列 X：长度 num_steps
* 标签序列 Y：就是 X 整体右移 1 个字符

举例来说：
* X = "nce of the"（10 个字符）
* Y = "ce of the "（把 X 往右挪一格）

所以 X 和 Y 的 shape 一定一样：都是 (batch_size, num_steps)（先别 one-hot）。

## 四、seq_data_iter_random（随机采样）

### 1. 在 “整本书展平后的 corpus” 里随机选起点

* 先在 corpus 里找很多个“可作为子序列起点的位置”
* 打乱这些起点
* 每次取 batch_size 个起点，截取长度为 num_steps 的片段当作 X
* 同样位置 +1 截取当作 Y

因此随机采样拿到的两条序列，通常来自 corpus 的完全不同位置，彼此不连续。

### 2. 举例来说
**假设文本有 400 个字符，batch_size=2，num_steps=10**
1. 起点数量 num_subseqs
    * `num_subseqs = (len(corpus) - 1) // num_steps `
    * 代入：(400 - 1) // 10 = 39
    * 这表示按每段长度 10 来 “分块”，最多能形成 39 条完整子序列样本（每条样本有 X 和 Y）。
2. 起点列表 initial_indices
    * [0, 10, 20, 30, ..., 380]
    * shuffle 打乱后，假设变成 [120, 40, 260, 0, 170, ...]
3. 每个 epoch 能产生的 batch
    * num_batches = num_subseqs // batch_size = 39 // 2 = 19
    * 所以一个 epoch 会输出 19 个 batch，总共用到：19 * 2 = 38 条子序列样本
    * 会剩下 1 条样本起点没被用到，但无所谓

## 五、seq_data_iter_sequential（顺序采样）
### 1. 顺序采样的核心思想
**先“切成 batch_size 行”，再沿列滑窗**
1. 把一维长序列 corpus 变成一个二维矩阵 Xs：Xs.shape = (batch_size, num_cols)
2. 每一行是一段连续文本
3. 然后在列方向用窗口长度 num_steps 依次切 batch

### 2. 举例来说
**假设文本有 400 个字符，batch_size=2，num_steps=10**
1. 选取 offset = random.randint(0, num_steps)，这主要是使得每个 epoch 从不同位置开始切，避免每轮边界完全一致。
2. 顺序采样要保证能 reshape 成 (batch_size, num_cols)，所以会丢掉一点尾巴：
    * 先取可整除部分长度：num_tokens = ((len(corpus) - offset - 1) // batch_size) * batch_size
    > 这里 -1 是为了给 Y（右移一位）留空间 \
    > 代入（举例 offset=3） \
    > len(corpus) - offset - 1 = 400 - 3 - 1 = 396 \
    > 396 // 2 = 198 \
    > num_tokens = 198 * 2 = 396
    * Xs = corpus[offset : offset + num_tokens]
    * Ys = corpus[offset + 1 : offset + 1 + num_tokens]
3. reshape 成二维
    * Xs.shape = (2, 198)
    * Ys.shape = (2, 198)
4. 在列方向滑窗生成每个 batch：(batch_size, num_steps)


## 六、RNN 网络输入
现在不管 random 还是 sequential，我们都已经拿到了：
* X：整数索引，shape (batch_size, num_steps)
* Y：整数索引，shape (batch_size, num_steps)

### 1. One-hot：加上 vocab 这一维
对 X 做 one-hot 后，shape：(batch_size, num_steps, vocab_size)
* 每条序列有 num_steps 个时间步
* 每个时间步用一个 27 维 one-hot 表示当前字符

### 2. 交换维度：把时间步放到最前面
将 X 变成 (num_steps, batch_size, vocab_size)
```python
for X_t in inputs:     # 迭代第0维
    # X_t: (batch_size, vocab_size)
    ...
```
* 每次循环取到的 $X_t$ 就是一个时间步的 batch 输入
* $X_t.shape$ = (batch_size, vocab_size)
* 正好能直接做：(batch_size, vocab_size) @ (vocab_size, num_hiddens)

如果不把时间步放在第 0 维，那要按时间步取 $X_t$ 就得写成：：
```python
for t in range(num_steps):
    X_t = inputs[:, t, :]  # (batch_size, vocab_size)
```
也可以运行，但是这样在 pytorch 取数组的时候就会出现性能问题

## 七、RNN 中的数据流
### 1. 输入、状态、参数的 shape
* 输入 X_t：shape = (batch_size, vocab_size) = (B, 27)
* 隐藏状态 H_t：shape = (batch_size, num_hiddens) = (B, H)
参数
* W_xh：shape = (vocab_size, num_hiddens) = (27, H)
* W_hh：shape = (num_hiddens, num_hiddens) = (H, H)
* b_h：shape = (num_hiddens,) = (H,)

### 2. 隐藏状态更新（以 tanh 为例）
$$H_t = tanh(X_t W_{xh} + H_{t-1} W_{hh} + b_h)$$
逐项看 shape：
* X_t @ W_xh： (B, 27) @ (27, H) = (B, H)
* H_{t-1} @ W_hh： (B, H) @ (H, H) = (B, H)
* +b_h： (B, H) + (H,) = (B, H) （广播机制）
* tanh：不改 shape → (B,H)

### 3. 输出层（预测下一个字符的 logits）
$$O_t = H_t W_{hq} + b_q$$
* W_hq：(H, vocab_size) = (H, 27)
* b_q：(vocab_size,) = (27,)
* O_t shape：(B,H) @ (H,27) = (B,27)

再 softmax 得到每个字符的概率分布：
* P_t shape：(B,27)